In [2]:
from get_samples import split_data
from sklearn import metrics
import normalize
import naivebayes

X_train, X_test, y_train, y_test = split_data()

not_normalized = (X_train, X_test)
mean_normalized = normalize.mean_normalization(not_normalized)
min_max_normalized = normalize.min_max_normalization(not_normalized)

# Run gaussian naive bayes

y_pred_not_normalized = naivebayes.run_naive_bayes(X_train, X_test, y_train, "gaussian")
y_pred_mean_normalized = naivebayes.run_naive_bayes(mean_normalized[0], mean_normalized[1], y_train, "gaussian")
y_pred_min_max_normalized = naivebayes.run_naive_bayes(min_max_normalized[0], min_max_normalized[1], y_train, "gaussian")

acc_g_not_normalized = metrics.accuracy_score(y_test, y_pred_not_normalized)
acc_g_mean_normalized = metrics.accuracy_score(y_test, y_pred_mean_normalized)
acc_g_min_max_normalized = metrics.accuracy_score(y_test, y_pred_min_max_normalized)

print(f"Accuracy Gaussian Naive Bayes not normalized: {acc_g_not_normalized}")
print(f"Accuracy Gaussian Naive Bayes mean normalized: {acc_g_mean_normalized}")
print(f"Accuracy Gaussian Naive Bayes min-max normalized: {acc_g_min_max_normalized}")

# Run complement naive bayes

y_pred_not_normalized = naivebayes.run_naive_bayes(not_normalized[0], not_normalized[1], y_train, "complement")
y_pred_min_max_normalized = naivebayes.run_naive_bayes(min_max_normalized[0], min_max_normalized[1], y_train, "complement")

acc_g_not_normalized = metrics.accuracy_score(y_test, y_pred_not_normalized)
acc_g_min_max_normalized = metrics.accuracy_score(y_test, y_pred_min_max_normalized)

print(f"Accuracy Complement Naive Bayes not normalized: {acc_g_not_normalized}")
print(f"Accuracy Complement Naive Bayes min-max normalized: {acc_g_min_max_normalized}")

# Run multinomial naive bayes

y_pred_not_normalized = naivebayes.run_naive_bayes(not_normalized[0], not_normalized[1], y_train, "multinomial")
y_pred_min_max_normalized = naivebayes.run_naive_bayes(min_max_normalized[0], min_max_normalized[1], y_train, "multinomial")

acc_g_not_normalized = metrics.accuracy_score(y_test, y_pred_not_normalized)
acc_g_min_max_normalized = metrics.accuracy_score(y_test, y_pred_min_max_normalized)

print(f"Accuracy Multinomial Naive Bayes not normalized: {acc_g_not_normalized}")
print(f"Accuracy Multinomial Naive Bayes min-max normalized: {acc_g_min_max_normalized}")

# Run bernoulli naive bayes

y_pred_not_normalized = naivebayes.run_naive_bayes(not_normalized[0], not_normalized[1], y_train, "bernoulli")
y_pred_mean_normalized = naivebayes.run_naive_bayes(mean_normalized[0], mean_normalized[1], y_train, "bernoulli")
y_pred_min_max_normalized = naivebayes.run_naive_bayes(min_max_normalized[0], min_max_normalized[1], y_train, "bernoulli")

acc_g_not_normalized = metrics.accuracy_score(y_test, y_pred_not_normalized)
acc_g_mean_normalized = metrics.accuracy_score(y_test, y_pred_mean_normalized)
acc_g_min_max_normalized = metrics.accuracy_score(y_test, y_pred_min_max_normalized)

print(f"Accuracy Bernoulli Naive Bayes not normalized: {acc_g_not_normalized}")
print(f"Accuracy Bernoulli Naive Bayes mean normalized: {acc_g_mean_normalized}")
print(f"Accuracy Bernoulli Naive Bayes min-max normalized: {acc_g_min_max_normalized}")

Accuracy Gaussian Naive Bayes not normalized: 0.3711513342041426
Accuracy Gaussian Naive Bayes mean normalized: 0.3692853144243329
Accuracy Gaussian Naive Bayes min-max normalized: 0.36872550849039
Accuracy Complement Naive Bayes not normalized: 0.27729053927971636
Accuracy Complement Naive Bayes min-max normalized: 0.27841015114760215
Accuracy Multinomial Naive Bayes not normalized: 0.3827206568389625
Accuracy Multinomial Naive Bayes min-max normalized: 0.3450270572868072
Accuracy Bernoulli Naive Bayes not normalized: 0.13472662810225788
Accuracy Bernoulli Naive Bayes mean normalized: 0.3676058966225042
Accuracy Bernoulli Naive Bayes min-max normalized: 0.13304721030042918
